In [36]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd
import time
import math

#Block chrome closes immediately after being launched
options = webdriver.ChromeOptions()
options.add_experimental_option("detach", True)
#Create chrome webdriver object
driver = webdriver.Chrome(options=options)

# 웹페이지 접근
url = 'http://kpat.kipris.or.kr/kpat/searchLogina.do?next=MainSearch'
driver.get(url)
time.sleep(1)

html = driver.page_source
# HTML 문자열을 파싱하여 BeautifulSoup 객체 생성
soup = BeautifulSoup(html, 'html.parser')

search = driver.find_element(By.CSS_SELECTOR, "#queryText") 

# 요소 클릭하기
search.click()

input_word=input("크롤링할 단어를 입력하세요")

# 순차적으로 키 입력하기
search.send_keys(input_word, Keys.ENTER)



In [47]:
html = driver.page_source
# HTML 문자열을 파싱하여 BeautifulSoup 객체 생성
soup = BeautifulSoup(html, 'html.parser')
특허s = soup.select('.search_section>article')
특허_list = []
# print(특허s)
for 특허 in 특허s:
    try:
        # 상태 
        status = 특허.select_one('h1.stitle a').text.strip()
        # 제목
        title = 특허.select_one('.search_section_title > h1 > a:nth-child(2)').text.strip()
    
        # # IPC
        # IPC  = 특허.select_one('.search_info_list .mainlist_topinfo li:nth-child(1) span.point01')

        # IPC 코드들
        IPC_elements = 특허.select('.search_info_list .mainlist_topinfo li:nth-child(1) span.point01')
        
        IPC_codes = [ipc.text.strip() for ipc in IPC_elements]
        IPC = ', '.join(IPC_codes)  # 여러 개의 IPC 코드를 콤마로 구분하여 문자열로 변환
        # print(IPC)
        # 출원번호(일자)
        numdate_element = 특허.select_one('li:nth-child(3) > span.point01 > a')
        numdate_text = numdate_element.text.strip()
        num, date = numdate_text.split(' (')
        date = date.rstrip(')')
    

        # 출원인 
        applicant = 특허.select_one('#mainsearch_info_list > div.mainlist_topinfo > li:nth-child(4) > a > font').text.strip()
        
        #최종권리자 
        holder = 특허.select_one('#mainsearch_info_list > div.mainlist_topinfo > li.left_width.letter1 > span.point01 > a > font')
        
        # citations
        try:
            citations  = 특허.select_one('#mainsearch_info_list > div.mainlist_topinfo > li:nth-child(6) > span.point01 > a').text.strip()
            # print(citations)
        except Exception as e:
            citations = 'N/A'
        # print(citations)
        
        # 요약
        abstract= 특허.select_one('#mainsearch_info_list > div.search_txt').text.strip()
        
        특허_list.append([status,title, IPC, num,date,applicant,holder,citations, abstract])
        
    except Exception as e:
        print(f"Error extracting data: {e}")
print(특허_list)

MaxRetryError: HTTPConnectionPool(host='localhost', port=9543): Max retries exceeded with url: /session/be3c0bd9dfb967273717800759e3e4a5/source (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001A2FFEC1220>: Failed to establish a new connection: [WinError 10061] 대상 컴퓨터에서 연결을 거부했으므로 연결하지 못했습니다'))

In [46]:
# 데이터프레임으로 변환 및 CSV 파일로 저장
df = pd.DataFrame(특허_list, columns=['Status', 'Title', 'IPC', 'Application Number', 'Application Date', 'Applicant', 'Holder', 'Citations', 'Abstract'])
df.to_csv('patent_data.csv', index=False)

# 브라우저 종료
driver.quit()

print("데이터 수집 완료")

데이터 수집 완료
